# Organization Activity on Wikipedia

The parameters in the cell below can be adjusted to explore other politicians and time frames.

### How to explore other organizations?
The ***organization_id*** is an internal identifier that connects the different social media accounts. You can [use this other notebook](../organizations.ipynb?autorun=true) to get other the identifiers of other politicians.

***Alternatively***, you can direcly use the [organizations API](http://mediamonitoring.gesis.org/api/organizations/swagger/), or access it with the [SMM Wrapper](https://pypi.org/project/smm-wrapper/).

## A. Set Up parameters

In [ ]:
# Parameters: 
organization_id = 219
from_date = '2017-09-01'
to_date = '2018-12-31'
aggregation = 'week'

## B. Using APIs
### B.1 Using the SMM Organization API

In [ ]:
library("httr")
library("jsonlite")

# prepare urls
base <- "http://mediamonitoring.gesis.org/api/organizations/"
url_chobs <- paste(base,"wikipedia/chobs/organizations/", organization_id, "/?", sep="")
url_organization <- paste(base,"all/", organization_id, "/", sep="")

# prepare parameters
params = list(
    from_date=from_date,
    to_date=to_date,
    aggregate_by=aggregation
)

# using the api to get the organization info and change objects
organization <- fromJSON(content(GET(url_organization), "text", encoding="UTF-8"), flatten = TRUE)
chobs = as.data.frame(fromJSON(content(GET(url_chobs, query=params), "text", encoding="UTF-8"), flatten = TRUE))

### B.2 Using the Wikiwho API

In [ ]:
# dplyr contains the summarise function
library('dplyr', warn.conflicts = FALSE)

#prepare urls
base_ww <- "https://www.wikiwho.net/de/edit_persistence/v1.0.0-beta/page/"

#prepare parameters
params = list(
    start=from_date,
    end=to_date 
)

# query the Wikiwho API for Wikipedia actions (edit_persistance)
wiki <- data.frame()
for (wp_id in organization$wp_ids) {
    json_wiki <- fromJSON(content(GET(paste(base_ww, wp_id ,"/?",sep=""), query=params), "text", encoding="UTF-8"))
    wiki <- rbind(wiki, json_wiki$editions)
}

# calculate undos
wiki$undos <- wiki$dels + wiki$reins

# calculate conflict score
wiki <- summarise(group_by(wiki, year_month),undos = sum(undos),
                  conflict = sum(conflict), elegibles = sum(elegibles))
wiki$conflict_score <- wiki$conflict/ wiki$elegibles

# format the dates
wiki$year_month<-as.POSIXct(strptime(wiki[["year_month"]], 
                                              format='%Y-%m-%d'),tz = 'GMT')

# replace missing with 0
wiki[is.na(wiki)] <- 0

### B.3 Using the Wikimedia API

In [ ]:
# prepare the urls
pre <- "https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/de.wikipedia.org/all-access/user/"
post <- paste("/daily/", gsub("-", "", from_date),"/", gsub("-", "", to_date), sep="")

# get the data throught the Wikimedia API
appended_views = list()
for (wp_title in organization$wp_titles) {
    x <- tryCatch({
        view = as.data.frame(fromJSON(content(GET(
            paste(pre, URLencode(wp_title, reserved=TRUE), post, sep="")), "text", 
            encoding="UTF-8"), flatten = TRUE))
        appended_views <- append(appended_views, list(view))
    }, error = function(e) e )
}
views  <- dplyr::bind_rows(appended_views)

# prepare the dates in the right format
views$items.timestamp<-as.POSIXct(strptime(views[["items.timestamp"]], 
                                           format='%Y%m%d%H'),tz = 'GMT')

# daily, weekly or monthly aggregation of the daa
views<-summarise(group_by(views, time = cut(items.timestamp, aggregation)), 
                 items.views = sum(items.views))

## C. Plotting
### C.1 Plot Wikipedia Activity

In [ ]:
library("ggplot2")
options(repr.plot.width=8, repr.plot.height=3)

# merge data so that the x scales are equal
wiki_data = merge( views, chobs, by.x='time', by.y='labels', all.x = TRUE)

#plotting wikipedia changes
ggplot(data = wiki_data, mapping = aes(as.Date(time))) +
  geom_line(aes(y = values), na.rm=TRUE) + 
  labs(title = "Wikipedia Activity", y = "Changes") +
  theme(axis.text.x = element_blank(), axis.title.x = element_blank()) +
  scale_x_date(date_breaks = "1 month")

#plotting wikipedia views
ggplot(data = wiki_data, mapping = aes(as.Date(time))) +
  geom_line(aes(y = items.views)) +   
  labs(y = "Views") +
  theme(axis.text.x = element_text(angle = 60, hjust = 0.5, vjust = 0.5),
        axis.title.x = element_blank()) +
  scale_x_date(date_breaks = "1 month")

### C.2 Plot Wikipedia Disagreement

In [ ]:
# plotting wikipedia undo actions
ggplot(data = wiki, mapping = aes(as.Date(year_month))) +
  geom_line(aes(y = undos)) +
  labs(title = "Wikipedia Disagreements", y = "Undos") +
  theme(axis.text.x = element_blank(), axis.title.x = element_blank()) +
  scale_x_date(date_breaks = "1 month")

# plotting conflict
ggplot(data = wiki, mapping = aes(as.Date(year_month))) +
  geom_line(aes(y = conflict_score)) +
  labs(y = "Conflict") +
  theme(axis.text.x = element_text(angle = 60, hjust = 0.5, vjust = 0.5),
        axis.title.x = element_blank()) +
  scale_x_date(date_breaks = "1 month")